# Web services with Flask

Content is based on these ressources:  
https://blog.miguelgrinberg.com/post/designing-a-restful-api-with-python-and-flask   
https://blog.miguelgrinberg.com/post/restful-authentication-with-flask

## To deploy flask to production env
This is outside the scope of this course   

https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-uswgi-and-nginx-on-ubuntu-18-04

# Flask intro
With Anaconda we got flask installed allready (check with `conda list` from inside your project folder).  
The following example code is the most basic implementation of a flask rest service:  
```python
#!flask/bin/python
from flask import Flask

app = Flask(__name__)

@app.route('/flask_app/')
def index():
    return "Hello, World from flask server!"

if __name__ == '__main__':
    app.run(debug=True)
```
Save it in a file: flask_app.py and run it from cli  
You can now access in browser from `http://127.0.0.1:5000/flask_app`

## Flask extended
Copy the file to flask_app_ext.py and add more http methods in the @app.route configuration

```python
#!flask/bin/python
from flask import Flask, jsonify, abort, request


app = Flask(__name__)

tasks = [
    {
        'id': 1,
        'title': 'Buy groceries',
        'description': 'Milk, Cheese, Pizza, Fruit, Tylenol', 
        'done': False
    },
    {
        'id': 2,
        'title': 'Learn Python',
        'description': 'Need to find a good Python tutorial on the web', 
        'done': False
    }
]


@app.route('/todo/api/tasks', methods=['GET'])
def get_tasks():
    return jsonify({'tasks': tasks})


@app.route('/todo/api/tasks/<int:task_id>', methods=['GET'])
def get_task(task_id):
    task = [task for task in tasks if task['id'] == task_id]
    if len(task) == 0:
        abort(404)
    return jsonify({'task': task[0]})

if __name__ == '__main__':
    app.run(debug=True)

@app.route('/todo/api/tasks', methods=['POST'])
def create_task():
    if not request.json or not 'title' in request.json:
        abort(400)
    task = {
        'id': tasks[-1]['id'] + 1,
        'title': request.json['title'],
        'description': request.json.get('description', ""),
        'done': False
    }
    tasks.append(task)
    return jsonify({'task': task}), 201

@app.route('/todo/api/tasks/<int:task_id>', methods=['PUT'])
def update_task(task_id):
    task = [task for task in tasks if task['id'] == task_id]
    if len(task) == 0:
        abort(404)
    if not request.json:
        abort(400)
    if 'title' in request.json and type(request.json['title']) != unicode:
        abort(400)
    if 'description' in request.json and type(request.json['description']) is not unicode:
        abort(400)
    if 'done' in request.json and type(request.json['done']) is not bool:
        abort(400)
    task[0]['title'] = request.json.get('title', task[0]['title'])
    task[0]['description'] = request.json.get('description', task[0]['description'])
    task[0]['done'] = request.json.get('done', task[0]['done'])
    return jsonify({'task': task[0]})

@app.route('/todo/api/tasks/<int:task_id>', methods=['DELETE'])
def delete_task(task_id):
    task = [task for task in tasks if task['id'] == task_id]
    if len(task) == 0:
        abort(404)
    tasks.remove(task[0])
    return jsonify({'result': True})

```

## Exercise
1. Create a restful webservice that can respond to the following requests:  

|Method|URL|Example|Response|Description|
|--|--|--|--|--|
|GET|/datagenerator/api/person/{no}|/datagenerator/api/person/100|[{"id":1,"name":"Holger"},{},{}]|Returns 100 person objects|
|POST|datagenerator/api/person|POST json: {"name":"Henrietta"}|{"id":101,"name":"Henrietta"}|Adds a new Person object to the list of persons on the server|

2. Use a Mysql table to store and retrieve Person objects

## Running flask server in production

When we run our flask server, we are actually running [Werkzeug's development WSGI server](https://palletsprojects.com/p/werkzeug/), and passing our flask app as the WSGI callable.

This development server is not intended for use in production. It is not designed to be particularly efficient, stable, or secure.

We must therefore replace the Werkzeug dev server with a production-ready WSGI server such as uWSGI when moving to production, no matter where the app will be available.

### WSGI standard interface
WSGI (pronounced **Whiskey**) stands for: Web Server Gateway Interface and is a simple calling convention for web servers to forward requests to web applications or frameworks written in Python  

- **WSGI** a set of specs to be followed by any WSGI server
- **uWSGI** a production ready server that upholds the WSGI specs and translates client requests to the application
- **uwsgi** a binary wire protocol for communication between uWSGI server and a full sized web server like nginx

The web server (uWSGI) sends requests to the application by triggering a defined “callable”.   
The callable is simply an entry point into the application where the web server can call a function with some parameters.   
The expected parameters are a dictionary of environmental variables and a callable provided by the web server (uWSGI) component.

In response, the application returns an iterable that will be used to generate the body of the client response.



## Install instructions on Digital Ocean
### Step 1 - 3: Initial environment setup and flask app

For digital ocean based on 
[This article steps 1 - 6](https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-uswgi-and-nginx-on-ubuntu-18-04)
```bash
sudo apt -y update
sudo apt -y upgrade
sudo apt -y install python3-pip python3-dev build-essential libssl-dev libffi-dev python3-setuptools
sudo apt -y install python3-venv
# make parent dir for flask app:
mkdir ~/flask_demo; cd ~/flask_demo
# Create a virtual environment for Flask project’s Python requirements
python3 -m venv flask_demo_env
# activate the virtual env (change should be reflected in prompt)
source flask_demo_env/bin/activate
# install wheel to ensure smooth installs (even if some dependencies are missing)
pip install wheel
# install flask and uwsgi server
pip install uwsgi flask
```
upload your flask_app.py file from earlier to your flask_demo folder on digital ocean. Change the last line from:  
`app.run(debug=True)` to `app.run(host='0.0.0.0')` So you can access the application over the internet.
In the digital ocean bash
```bash
# open the firewall to port 5000
sudo ufw allow 5000
# test the flask server app.py
python flask_app.py
```
Now go to a browser and check out <your ip>:5000 to see your first flask server running.  
Go back to digital ocean and do. `<ctrl> C` to terminate the application. And `deactivate` to deactivate the virtual environment (Now any use of python will be on the system default installation, so be carefull)

### step 4: WSGI entry point
Inside folder: flask_demo create file: wsgi.py:
```python
from flask_app import app

if __name__ == "__main__":
    app.run()
```
Test the entry point from bash to see if uwsgi can serve the application:

`uwsgi --socket 0.0.0.0:5000 --protocol=http -w wsgi:app`

Test in browser `http://<your ip>:5000` 

### WSGI configuration

Create a new file: flask_app.ini inside the flask_demo folder
```sh
# tell the uWSGI server to apply these settings
[uwsgi]
# module is the name of the wsgi.py file (without the py) plus the callable within the file
module = wsgi:app

# start in master mode with 5 processes
master = true
processes = 5

# use a unix socket for interprocess communication
socket = flask_app.sock
chmod-socket = 660
vacuum = true

die-on-term = true
# no protocol specified since uwsgi binary protocol is default and understood by nginx
```

### Step 5: Systemd Unit file
To start the app on server boot
Go to `/etc/systemd/system` and create file: flask_demo.service:
```sh
[Unit]
Description=uWSGI instance to serve flask_demo
# Only start app after is network up and running
After=network.target

[Service]
User=<your user>
# This group gives ease access for nginx to communicate with uWSGI processes.
Group=www-data

# Set working directory and the PATH environmental variable so the ubuntu init system can locate the executables
WorkingDirectory=/home/<your user>/flask_demo
Environment="PATH=/home/<your user>/flask_demo/flask_demo_env/bin"
ExecStart=/home/<your user>/flask_demo/flask_demo_env/bin/uwsgi --ini flask_app.ini

# Install section tells systemd to start when the regular multi-user system is up and running:
[Install]
WantedBy=multi-user.target
```
Save and close
```sh
sudo systemctl start flask_demo
sudo systemctl enable flask_demo
sudo systemctl status flask_demo

```

### Step 6: Nginx
Modify the file: `sudo nano /etc/nginx/sites-available/default`:
And in the `server {...}` section add a new location:
```yml
server {
    ...
    location /flask_app/ {
        include uwsgi_params;
        uwsgi_pass unix:/home/<your user>/flask_demo/flask_app.sock;
    }
}
```
Test syntax errors on nginx: `sudo nginx -t`   
Restart nginx: `sudo systemctl restart nginx`   


Test in browser: https://domain_name/flask_app

### Troubleshooting:

`sudo less /var/log/nginx/error.log`: checks the Nginx error logs.  
`sudo less /var/log/nginx/access.log`: checks the Nginx access logs.  
`sudo journalctl -u nginx`: checks the Nginx process logs.  
`sudo journalctl -u flask_demo`: checks your Flask app’s uWSGI logs.  
`sudo systemctl restart flask_demo`: restarts the service we made (do this after changes made to flask_app.py)  
`sudo systemctl restart nginx`: restarts nginx (do this after changes to `/etc/nginx/sites-enabled/default`)  
